In [22]:
import random
import torch.nn as nn
import torch
import pickle
import pandas as pd
from pandas import Series, DataFrame
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, matthews_corrcoef, f1_score, precision_score, recall_score
import numpy as np
import torch.optim as optim
folder = "/../models/"
import matplotlib.pyplot as plt
from vocabulary import Vocabulary
from dataset import Dataset
from dataset import collate_fn_no_activity as collate_fn
from models import Generator
from tqdm.autonotebook  import trange, tqdm
import os
from collections import defaultdict
import sys
import json

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
SEED = 12456789
np.random.seed(SEED)
random.seed(SEED)

In [3]:
def kills_(row, prey):
    targets = row.targetSpecies
    for target in targets:
        if prey in target:
            return True    
    return False

In [4]:
db_path_ac = folder+"pickles/daasp_with_activities.pkl"
df_dbaasp_ac = pd.read_pickle(db_path_ac)
db_path_in = folder+"pickles/daasp_inactives_with_activities.pkl"
df_dbaasp_in = pd.read_pickle(db_path_in)
df_dbaasp = pd.concat([df_dbaasp_ac, df_dbaasp_in])

In [5]:
group = df_dbaasp.groupby(["ID","Name", "N terminus", "Sequence", "C terminus"], as_index=False)
df_dbaasp = pd.DataFrame(group["targetSpecies"].aggregate(list))
df_dbaasp["HeLa"] =  df_dbaasp.parallel_apply(lambda x: kills_(x, "HeLa"), axis=1)
forhela = df_dbaasp[df_dbaasp["HeLa"]==True]

In [8]:
seq_forhela = forhela.Sequence.to_list()

In [9]:
def inforhela(seq):
    if seq in seq_forhela:
        return True
    else:
        return False

# Load data

In [10]:
df = pd.read_pickle(folder + "pickles/all_sequences_with_NN_prop_helicity.pkl")

In [11]:
df["HeLa"] = df.Sequence.map(inforhela)

In [27]:
df.to_pickle(folder + "pickles/DAASP_RNN_dataset_with_anticancer.plk")

In [12]:
df_training = df.query("Set == 'training' and HeLa == True")
df_test = df.query("Set == 'test' and HeLa == True")

In [19]:
hela = df.query("HeLa == True and (Set == 'training' or Set == 'test') and activity == True")

In [20]:
len(hela)

53

In [21]:
hela.to_csv("data/TL.csv", index=False)

In [13]:
len(df_test[df_test["activity"]==True])

16

In [14]:
len(df_training[df_training["activity"]==True])

37

In [14]:
df_training = df.query("Set == 'training' and HeLa == True")
df_test = df.query("Set == 'test' and HeLa == True")

if torch.cuda.is_available():
    device = "cuda" 
else:
    device = "cpu" 

In [15]:
print("Against HeLa:\nactive training "+ str(len(df_training[df_training["activity"]==1])) \
      + "\nactive test " + str(len(df_test[df_test["activity"]==1])) \
      + "\ninactive training "+ str(len(df_training[df_training["activity"]==0])) \
      + "\ninactive test " + str(len(df_test[df_test["activity"]==0])))

Against HeLa:
active training 37
active test 16
inactive training 22
inactive test 14


# Define helper functions

In [16]:
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return category_i

def nan_equal(a,b):
    try:
        np.testing.assert_equal(a,b)
    except AssertionError:
        return False
    return True

def models_are_equal(model1, model2):
    model1.vocabulary == model2.vocabulary
    model1.hidden_size == model2.hidden_size
    for a,b in zip(model1.model.parameters(), model2.model.parameters()):
        if nan_equal(a.detach().numpy(), b.detach().numpy()) == True:
            print("true")

# Define hyper parameters

In [17]:
n_embedding  = 100
n_hidden = 400
n_layers = 2
n_epoch = 1500
learning_rate = 0.00001
momentum = 0.9
batch_size = 10
epoch = 22

# Loading and Training

In [18]:
if not os.path.exists(folder+"pickles/generator_TL_anticancer_results.pkl"):
    
    model = Generator.load_from_file(folder+"models/RNN-generator/ep{}.pkl".format(epoch))
    model.to(device)
    vocabulary = model.vocabulary

    df_training_active = df_training.query("activity == 1")
    df_test_active = df_test.query("activity == 1")
    df_training_inactive = df_training.query("activity == 0")
    df_test_inactive = df_test.query("activity == 0")

    training_dataset_active = Dataset(df_training_active, vocabulary, with_activity=False)
    test_dataset_active = Dataset(df_test_active, vocabulary, with_activity=False)
    training_dataset_inactive = Dataset(df_training_inactive, vocabulary, with_activity=False)
    test_dataset_inactive = Dataset(df_test_inactive, vocabulary, with_activity=False)

    
    optimizer = optim.SGD(model.model.parameters(), lr = learning_rate, momentum=momentum)

    # the only one used for training
    training_dataloader_active = torch.utils.data.DataLoader(training_dataset_active, batch_size=batch_size, shuffle=True, collate_fn = collate_fn, drop_last=True, pin_memory=True, num_workers=4)

    # used for evaluation
    test_dataloader_active = torch.utils.data.DataLoader(test_dataset_active, batch_size=batch_size, shuffle=False, collate_fn = collate_fn, drop_last=False, pin_memory=True, num_workers=4)
    training_dataloader_inactive = torch.utils.data.DataLoader(training_dataset_inactive, batch_size=batch_size, shuffle=False, collate_fn = collate_fn, drop_last=False, pin_memory=True, num_workers=4)
    test_dataloader_inactive = torch.utils.data.DataLoader(test_dataset_inactive, batch_size=batch_size, shuffle=False, collate_fn = collate_fn, drop_last=False, pin_memory=True, num_workers=4)
    training_dataloader_active_eval = torch.utils.data.DataLoader(training_dataset_active, batch_size=batch_size, shuffle=False, collate_fn = collate_fn, drop_last=False, pin_memory=True, num_workers=4)

    training_dictionary = {}

    for e in trange(1, n_epoch + 1):
        print("Epoch {}".format(e))
        for i_batch, sample_batched in tqdm(enumerate(training_dataloader_active), total=len(training_dataloader_active) ):

            seq_batched = sample_batched[0].to(model.device, non_blocking=True) 
            seq_lengths = sample_batched[1].to(model.device, non_blocking=True)

            nll = model.likelihood(seq_batched, seq_lengths)

            loss = nll.mean()

            optimizer.zero_grad()
            loss.backward()  
            torch.nn.utils.clip_grad_value_(model.model.parameters(), 2)
            optimizer.step()

        model.save(folder+"models/RNN-generator-TL-ancticancer/anticancer_ep{}.pkl".format(e))


        print("\tExample Sequences")
        sampled_seq = model.sample(5)
        for s in sampled_seq:
            print("\t\t{}".format(model.vocabulary.tensor_to_seq(s, debug=True)))

        nll_training = []
        with torch.no_grad():
            for i_batch, sample_batched in enumerate(training_dataloader_active_eval):    
                seq_batched = sample_batched[0].to(model.device, non_blocking=True) 
                seq_lengths = sample_batched[1].to(model.device, non_blocking=True) 

                nll_training += model.likelihood(seq_batched, seq_lengths)

        nll_training_active_mean = torch.stack(nll_training).mean().item()
        print("\tNLL Train Active: {}".format(nll_training_active_mean))
        del nll_training

        nll_test = []
        with torch.no_grad():
            for i_batch, sample_batched in enumerate(test_dataloader_active):    
                seq_batched = sample_batched[0].to(model.device, non_blocking=True) 
                seq_lengths = sample_batched[1].to(model.device, non_blocking=True) 

                nll_test += model.likelihood(seq_batched, seq_lengths)

        nll_test_active_mean = torch.stack(nll_test).mean().item()
        print("\tNLL Test Active: {}".format(nll_test_active_mean))
        del nll_test

        nll_training = []
        with torch.no_grad():
            for i_batch, sample_batched in enumerate(training_dataloader_inactive):    
                seq_batched = sample_batched[0].to(model.device, non_blocking=True) 
                seq_lengths = sample_batched[1].to(model.device, non_blocking=True) 

                nll_training += model.likelihood(seq_batched, seq_lengths)

        nll_training_inactive_mean = torch.stack(nll_training).mean().item()
        print("\tNLL Train Inactive: {}".format(nll_training_inactive_mean))
        del nll_training

        nll_test = []
        with torch.no_grad():
            for i_batch, sample_batched in enumerate(test_dataloader_inactive):    
                seq_batched = sample_batched[0].to(model.device, non_blocking=True) 
                seq_lengths = sample_batched[1].to(model.device, non_blocking=True) 

                nll_test += model.likelihood(seq_batched, seq_lengths)

        nll_test_inactive_mean = torch.stack(nll_test).mean().item()
        print("\tNLL Test Inactive: {}".format(nll_test_inactive_mean))
        del nll_test
        print()

        training_dictionary[e]=[nll_training_active_mean, nll_test_active_mean, nll_training_inactive_mean, nll_test_inactive_mean]
    
    with open(folder+"pickles/generator_TL_anticancer_results.pkl",'wb') as fd:
        pickle.dump(training_dictionary, fd)
    
else:
    with open(folder+"pickles/generator_TL_anticancer_results.pkl",'rb') as fd:
        training_dictionary = pickle.load(fd)

min_nll_test_active = float("inf")
for epoch, training_values in training_dictionary.items():
    nll_test_active = training_values[1]

    if nll_test_active < min_nll_test_active:
        best_epoch = epoch
        min_nll_test_active = nll_test_active

Epoch 1



	Example Sequences
		<start>GRKKRRQRRRRAAAFKQEAYM<end><pad><pad><pad><pad><pad>
		<start>YKYPLRPRPRRIYNR<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>KWKSFLKTFKSAKKTKLHTALKAISS<end>
		<start>GILKTIKIAAKVLTTGLPALISWK<end><pad><pad>
		<start>FFGTLLKLAGKLL<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
	NLL Train Active: 15.809674263000488
	NLL Test Active: 20.997760772705078
	NLL Train Inactive: 28.6232967376709
	NLL Test Inactive: 39.77482223510742

Epoch 2



	Example Sequences
		<start>RLRRLIRKLRRRLRRLRRRLRR<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>RGRLSKIGQKIKCFVKKVRGVFK<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>LLRRILKRVVRILKRLYRRY<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>SWLRDWLNWASAMGSAKALGQRSGKSHKDKNLQPKYEQQCCQHEFPCVYCPEGYECPKH<pad>FTHSHGKH<end>
		<start>ENLLKRIRTLLR<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p


	Example Sequences
		<start>ILPWKWPWLPRR<end><pad><pad><pad><pad><pad><pad><pad>
		<start>GMWSKILGHLIR<end><pad><pad><pad><pad><pad><pad><pad>
		<start>GLLKRIKKLL<end><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>GWDWFKKWNNKRK<end><pad><pad><pad><pad><pad><pad>
		<start>FWGAIWKGIKGVAIAWGKA<end>
	NLL Train Active: 15.633214950561523
	NLL Test Active: 20.84632110595703
	NLL Train Inactive: 28.599773406982422
	NLL Test Inactive: 39.74785614013672

Epoch 4



	Example Sequences
		<start>VKRRKKHRCRVYNNGLPTGLYLRWC<end>
		<start>GLLSSLAKKALKQALAQW<end><pad><pad><pad><pad><pad><pad><pad>
		<start>VRRRPRPPYLPRPRPP<end><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>RPGYGQPYGPRGPTHPRL<end><pad><pad><pad><pad><pad><pad><pad>
		<start>FKKILRAWPRRILCILR<end><pad><pad><pad><pad><pad><pad><pad><pad>
	NLL Train Active: 15.520550727844238
	NLL Test Active: 20.74642562866211
	NLL Train Inactive: 28.586681365966797
	NLL Test Inactive: 39.72887420654297

Epoch 5


...


	Example Sequences
		<start>KKVLKKLTRMISWIKRKR<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>KWKSFLKTFKSAAKTVLHTALKAISS<end><pad><pad><pad><pad>
		<start>KWKSFLKTFKSAAKTFAKTVLHTALKAISS<end>
		<start>RWFRIQLQIRRWRNRR<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>KWKSFLKTFKSLKKTVLHTLLkAISS<end><pad><pad><pad><pad>
	NLL Train Active: 5.201077938079834
	NLL Test Active: 17.437082290649414
	NLL Train Inactive: 29.013681411743164
	NLL Test Inactive: 39.2170524597168

Epoch 1497



	Example Sequences
		<start>VNWKKIILGKIIKVVK<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>WLKGLKKIIKVASKMI<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>VNWKKILGKIIKVVK<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>KWKSFlKTFKSlKKTVLHTLLKAISS<end>
		<start>KWKSFLKTFKSLKKTVLHTLLKAISS<end>
	NLL Train Active: 5.200212001800537
	NLL Test Active: 17.436670303344727
	NLL Train Inactive: 29.016630172729492
	NLL Test Inactive: 39.21961212158203

Epoch 1498



	Example Sequences
		<start>VNWKKILGKIIKVVK<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>GFMASLAKKVL<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>KWKSFLKTFKSLKKTVLHTLLKAISS<end>
		<start>KWKSFLKTFKSAAKTVLHTALKAISS<end>
		<start>KWKSFLKTFKSlKKTVLHTLLKAISS<end>
	NLL Train Active: 5.199407577514648
	NLL Test Active: 17.436687469482422
	NLL Train Inactive: 29.0208797454834
	NLL Test Inactive: 39.221923828125

Epoch 1499



	Example Sequences
		<start>GFKDLLKGAAKTVLHAALKAISS<end><pad><pad><pad>
		<start>KWKSFlkTFKSlKKTVLHTLLKAISS<end>
		<start>KWKSFLKTFKSLKKTVLHTLLKAISS<end>
		<start>KWKSFlKTFKSFKKTVLHTLLKAISS<end>
		<start>KWKSFLKTFKSlKKTVLHTlLKAISS<end>
	NLL Train Active: 5.198610305786133
	NLL Test Active: 17.437835693359375
	NLL Train Inactive: 29.021968841552734
	NLL Test Inactive: 39.22259521484375

Epoch 1500



	Example Sequences
		<start>RWFKIQLQIRRRWRNKK<end><pad><pad>
		<start>GFKMALKLLKKVL<end><pad><pad><pad><pad><pad><pad>
		<start>WKWKKWLKKVL<end><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>VNWKKILGKIKKVVK<end><pad><pad><pad><pad>
		<start>VDKKPYRPRPRPPRRIYNR<end>
	NLL Train Active: 5.197785377502441
	NLL Test Active: 17.438697814941406
	NLL Train Inactive: 29.024099349975586
	NLL Test Inactive: 39.223114013671875




# Sampling evaluation

In [19]:
print(best_epoch)
model = Generator.load_from_file(folder+"models/RNN-generator-TL-ancticancer/anticancer_ep{}.pkl".format(best_epoch))

562


In [20]:
training_seq = df_training.Sequence.values.tolist()

def _sample(model, n):
    sampled_seq = model.sample(n)
    sequences = []
    for s in sampled_seq:
        sequences.append(model.vocabulary.tensor_to_seq(s))
    return sequences

def novelty(seqs, list_):
    novel_seq = []
    for s in seqs:
        if s not in list_:
            novel_seq.append(s)
    return novel_seq, (len(novel_seq)/len(seqs))*100

def is_in_training(seq, list_ = training_seq):
    if seq not in list_:
        return False
    else:
        return True

def uniqueness(seqs):
    unique_seqs = defaultdict(int)
    for s in seqs:
        unique_seqs[s] += 1
    return unique_seqs, (len(unique_seqs)/len(seqs))*100

In [21]:
# sample
seqs = _sample(model, 50000)
unique_seqs, perc_uniqueness = uniqueness(seqs)
notintraining_seqs, perc_novelty = novelty(unique_seqs, training_seq)
print(perc_uniqueness, perc_novelty)

# create dataframe
df_generated = pd.DataFrame(list(unique_seqs.keys()), columns =['Sequence']) 
df_generated["Repetition"] = df_generated["Sequence"].map(lambda x: unique_seqs[x])
df_generated["inTraining"] = df_generated["Sequence"].map(is_in_training)
df_generated["Set"] = "generated-TL-anticancer"

# save
df_generated.to_pickle(folder+"pickles/Generated-TL-anticancer.pkl")

61.958 99.89024823267376


61.958 99.89024823267376